# Import & Setup

In [1]:
import sys
sys.path.append(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/My-Package")

import pandas as pd
import numpy as np
import h3

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from shapely.geometry import Polygon

import statsmodels.api as sm

import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv(filepath_or_buffer=r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021 - One Row per Stop.csv",
                 index_col='Stop ID')

C:\Users\mikha\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (18,19,20,21,22,23,32,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df['Search Race'] = np.where(df['Search Conducted']==1, df['Race'], np.nan)
df['Arrest Race'] = np.where(df['Arrest']==1, df['Race'], np.nan)
df['Citation Race'] = np.where(df['Citation']==1, df['Race'], np.nan)

## Replace cols with Sparse-filled cols

In [4]:
sparse_cols = df.filter(like='Sparse').columns

# replace non sparse-filled cols with sparse-filled cols
for sparse_col in sparse_cols:
    df[sparse_col.replace(' - Sparse Filled', '')] = df[sparse_col]
    
# drop now-redundant cols
df.drop(columns=sparse_cols, inplace=True)

# Constants

## H3 Encoding

* Resolution options:
  * Official list w/ area https://h3geo.org/docs/core-library/restable/
  * Visualize at https://observablehq.com/@four43/h3-index-visualizer
* Looks like resolution=(7 or 8) is the way to go

In [5]:
RESOLUTIONS = [6,7,8,9,10]

race_set = sorted([race for race in df['Race'].unique().tolist() if race!='NATIVE AMERICAN'])

# Functions

## Create per cols

In [6]:
def create_PER_cols(dataframe:pd.DataFrame, numerator_cols:list, denominator_cols:list) -> None:
    for num_col in numerator_cols:
        for denom_col in denominator_cols:
            dataframe[f'{num_col} per {denom_col}'] = dataframe[num_col] / dataframe[denom_col]

## Race agg fncs

In [29]:
race_count_fncs = []

for race in race_set:
    # count_fnc_name = f'{race}'
    exec(f'def {race}(series): return (series=="{race}").sum()')
    race_count_fncs.append(eval(race))

# def count_ASIAN_stops(series):
#     return (series=='ASIAN').sum()

# def count_BLACK_stops(series):
#     return (series=='BLACK').sum()

# def count_HISPANIC_stops(series):
#     return (series=='HISPANIC').sum()

# def count_NATIVE_AMERICAN_stops(series):
#     return (series=='NATIVE AMERICAN').sum()

# def count_OTHER_stops(series):
#     return (series=='OTHER').sum()

# def count_WHITE_stops(series):
#     return (series=='WHITE').sum()

In [30]:
race_count_fncs

[<function __main__.ASIAN(series)>,
 <function __main__.BLACK(series)>,
 <function __main__.HISPANIC(series)>,
 <function __main__.OTHER(series)>,
 <function __main__.WHITE(series)>]

### Rename agg cols

In [77]:
def rename_agg_cols(columns:list) -> list:
    """ugly but necessary manual renaming of aggregated columns"""
    new_cols = []
    for col in columns:
        if type(col) == tuple and (col[1]=='sum'):
            new_cols.append(col[0])
        elif len(col)==2:
            if col[0] == 'Race':
                new_cols.append(f'Stops - {col[1]}')
            elif col[0] == 'Search Race':
                new_cols.append(f'Search Conducted - {col[1]}')
            elif col[0] == 'Citation Race':
                new_cols.append(f'Citation - {col[1]}')
            elif col[0] == 'Arrest Race':
                new_cols.append(f'Arrest - {col[1]}')
        else:
            if type(col) != tuple:
                new_cols.append(col)
            else:
                raise ValueError(col, "what's this col?")
    
    return new_cols

### Create race % of total columns

In [93]:
def create_race_pct_total_cols(dataframe:pd.DataFrame, metric_types:list) -> pd.DataFrame:
    """
    'metric_types' == ['Stops', 'Arrests'], etc.
    :: requires dataframe[metric_type] to exist (e.g., 'Stops' is total stops in that cell)
    :: requires dataframe[f'{metric_type} - {race}'] to exist for all races in race_set (e.g., 'Stops - WHITE' is a col)
    
    :: returns new pct columns
    """
    pct_total_cols = []
    for metric_type in metric_types:
        
        new_pct_cols = dataframe.filter(like=f'{metric_type} -').div(dataframe[metric_type], axis='index')
        
        new_pct_cols.columns = [f'{metric_type} - % Total - ' + col.split(' - ')[1] for col in new_pct_cols.columns]
        
        pct_total_cols.append(new_pct_cols)
        
    # return pct_total_cols
    return pd.concat(pct_total_cols, axis='columns')

## Create grouped dataframes

### OG

In [101]:
def create_grouped_dataframes(dataframe:pd.DataFrame, resolutions:list, numerator_cols:list, denominator_cols:list) -> None:
    """
    Creates 'grouped_dataframes' dict 
    Note that numerator_cols and denominator_cols should all be bools (for aggregation to work properly)
    """
    
    num_agg_dict = {col:'sum' for col in numerator_cols}
    denom_agg_dict = {col:'sum' for col in denominator_cols}
    race_count_dict = {}# {'Race': race_count_fncs} #{'Race':[count_ASIAN_stops, count_BLACK_stops, count_HISPANIC_stops, count_NATIVE_AMERICAN_stops, count_OTHER_stops, count_WHITE_stops]}
    # remaining_cols = list(set(dataframe.columns) - set(num_agg_dict.keys()) - set(denom_agg_dict.keys()))
    # other_cols_agg = {'Stops':pd.NamedAgg(column="Latitude", aggfunc='count')}
    
    aggregation_dict = {**num_agg_dict, **denom_agg_dict, **race_count_dict}
    
    # only count stops that occurred in MC (based on recorded Longitude & Latitude)
    in_mc_mask = ((38.915292 <= dataframe['Latitude']) & (dataframe['Latitude'] <= 39.414658)
                  & (-77.554198 <= dataframe['Longitude']) & (dataframe['Longitude'] <= -76.864813))
    dataframe = dataframe[in_mc_mask]
    
    # global grouped_dataframes
    grouped_dataframes = dict()
    for resolution in resolutions:
        print(resolution)
                
        grouped_dataframes[resolution] = dataframe.groupby(by=f'H3 Encoding - Res={resolution}').agg(aggregation_dict)
        other_col_grouped = dataframe.groupby(by=f'H3 Encoding - Res={resolution}').agg(
            Stops=pd.NamedAgg(column='Latitude', aggfunc='count'))
        
        grouped_dataframes[resolution] = pd.concat([grouped_dataframes[resolution], other_col_grouped], axis='columns')
        
        # add geo-coordinates
        grouped_dataframes[resolution]['Longitude'] = grouped_dataframes[resolution].index.map(lambda x: h3.h3_to_geo(x)[0])
        grouped_dataframes[resolution]['Latitude'] = grouped_dataframes[resolution].index.map(lambda x: h3.h3_to_geo(x)[1])
        
        # return grouped_dataframes
        # Create "... Per Stop", "... Per Citation"
        create_PER_cols(dataframe=grouped_dataframes[resolution],
                        numerator_cols=numerator_cols + ['Stops'],
                        denominator_cols=denominator_cols)
        
        grouped_dataframes[resolution].replace([np.inf, -np.inf], np.nan, inplace=True)

        return grouped_dataframes


num_cols = ['Citation', 'Arrest', 'Search Conducted'] # these should be actions the police can decide on - e.g., stopping somebody
denom_cols = ['Fatal', 'Alcohol', 'Accident', 'Personal Injury', 'Property Damage'] # these should be "negative" traffic events that simply occur - e.g., accidents, fatalities
grouped_dataframes = create_grouped_dataframes(dataframe=df,
                          resolutions=RESOLUTIONS, 
                          numerator_cols=num_cols, denominator_cols=denom_cols)

### New

In [130]:
def create_grouped_dataframes2(dataframe:pd.DataFrame, resolutions:list, numerator_cols:list, denominator_cols:list) -> None:
    """
    Creates 'grouped_dataframes' dict 
    Note that numerator_cols and denominator_cols should all be bools (for aggregation to work properly)
    """
    
    num_agg_dict = {col:'sum' for col in numerator_cols}
    denom_agg_dict = {col:'sum' for col in denominator_cols}
    race_count_dict = {race_col: race_count_fncs for race_col in ['Race',
                                                                  'Search Race',
                                                                  'Citation Race',
                                                                  'Arrest Race']
                       }
    # other_cols_agg = {'Stops':pd.NamedAgg(column="Latitude", aggfunc='count')}
    
    aggregation_dict = {**num_agg_dict, **denom_agg_dict, **race_count_dict}
    
    # only count stops that occurred in MC (based on recorded Longitude & Latitude)
    in_mc_mask = ((38.915292 <= dataframe['Latitude']) & (dataframe['Latitude'] <= 39.414658)
                  & (-77.554198 <= dataframe['Longitude']) & (dataframe['Longitude'] <= -76.864813))
    dataframe = dataframe[in_mc_mask]
    
    # global grouped_dataframes
    grouped_dataframes = dict()
    for resolution in resolutions:
        print(resolution)
                
        grouped_dataframes[resolution] = dataframe.groupby(by=f'H3 Encoding - Res={resolution}').agg(aggregation_dict)
        
        # why am I doing it like this..? just put it in the agg dict ffs
        other_col_grouped = dataframe.groupby(by=f'H3 Encoding - Res={resolution}').agg(
            Stops=pd.NamedAgg(column='Latitude', aggfunc='count'))
        
        grouped_dataframes[resolution] = pd.concat([grouped_dataframes[resolution], other_col_grouped], axis='columns')
        
        # add geo-coordinates
        grouped_dataframes[resolution]['Longitude'] = grouped_dataframes[resolution].index.map(lambda x: h3.h3_to_geo(x)[0])
        grouped_dataframes[resolution]['Latitude'] = grouped_dataframes[resolution].index.map(lambda x: h3.h3_to_geo(x)[1])
        
        # rename columns
        grouped_dataframes[resolution].columns = rename_agg_cols(grouped_dataframes[resolution].columns)
        
        # add % of total columns
        pct_cols = create_race_pct_total_cols(dataframe=grouped_dataframes2[resolution], 
                                              metric_types=['Stops','Arrest','Citation','Search Conducted'])
        grouped_dataframes[resolution] = pd.concat([grouped_dataframes2[resolution], pct_cols], axis='columns')
        
        # # return grouped_dataframes
        # # Create "... Per Stop", "... Per Citation"
        create_PER_cols(dataframe=grouped_dataframes[resolution],
                        numerator_cols=numerator_cols + ['Stops'],
                        denominator_cols=denominator_cols)
        
        # Black + Hispanic Metrics
        for metric_type in ['Stops','Arrest','Citation','Search Conducted']:
            try:
                grouped_dataframes2[resolution][f'BLACK + HISPANIC {metric_type} - % Total'] = grouped_dataframes2[resolution][f'{metric_type} - % Total - BLACK'] + grouped_dataframes2[resolution][f'{metric_type} - % Total - HISPANIC']
            except:
                print(f"Couldn't make 'BLACK + HISPANIC {metric_type} - % Total' col")
        
        # replace inf
        grouped_dataframes[resolution].replace([np.inf, -np.inf], np.nan, inplace=True)
        
    return grouped_dataframes


num_cols = ['Citation', 'Arrest', 'Search Conducted'] # these should be actions the police can decide on - e.g., stopping somebody
denom_cols = ['Fatal', 'Alcohol', 'Accident', 'Personal Injury', 'Property Damage'] # these should be "negative" traffic events that simply occur - e.g., accidents, fatalities
grouped_dataframes2 = create_grouped_dataframes2(dataframe=df,
                          resolutions=RESOLUTIONS, 
                          numerator_cols=num_cols, denominator_cols=denom_cols)

6
Couldn't make 'BLACK + HISPANIC Search Conducted - % Total' col
7
Couldn't make 'BLACK + HISPANIC Search Conducted - % Total' col
8
Couldn't make 'BLACK + HISPANIC Search Conducted - % Total' col
9
Couldn't make 'BLACK + HISPANIC Search Conducted - % Total' col
10
Couldn't make 'BLACK + HISPANIC Search Conducted - % Total' col


## Total #Cells

In [99]:
for res in RESOLUTIONS:
    num_cells = len(grouped_dataframes2[res])
    square_size = np.sqrt(len(grouped_dataframes2[res])).round(0)
    print(f'Resolution {res} has {num_cells} many cells (approx {square_size} X {square_size})')

Resolution 6 has 35 many cells (approx 6.0 X 6.0)
Resolution 7 has 141 many cells (approx 12.0 X 12.0)
Resolution 8 has 588 many cells (approx 24.0 X 24.0)
Resolution 9 has 2706 many cells (approx 52.0 X 52.0)
Resolution 10 has 11153 many cells (approx 106.0 X 106.0)


# Analysis

# Scatterplots

In [140]:
def regress_stops_per_acc_black_hisp_stops(resolution):
    mod = smf.wls(formula='''Q("Stops per Accident") ~ Q("BLACK + HISPANIC Stops - % Total")
                  ''', 
                  data=grouped_dataframes2[resolution],
                  weights=grouped_dataframes2[resolution]['Stops'])
    
    res = mod.fit(cov_type='HC3')
    print(f'!!!   {resolution}   !!!')
    display(res.summary())

## Black + Hispanic stops as % total vs stops/accident

In [141]:
for res in RESOLUTIONS:
   regress_stops_per_acc_black_hisp_stops(resolution=res)
   
   px.scatter(data_frame=grouped_dataframes2[res].query('Stops > 100'),
               x='BLACK + HISPANIC Stops - % Total',
               y='Stops per Accident',
               size='Stops',
            #    color='Stops - % Total - WHITE',
               trendline='ols',
               title=res).show()

!!!   6   !!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.006
Model:                                 WLS   Adj. R-squared:                 -0.039
Method:                      Least Squares   F-statistic:                   0.03279
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):              0.858
Time:                             19:26:31   Log-Likelihood:                -128.51
No. Observations:                       24   AIC:                             261.0
Df Residuals:                           22   BIC:                             263.4
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                47.5652     24.911      1.909      0.056      -1.259      96.389
Q("BLACK + HISPANIC Stops - % Total")     9.8978     54.664      0.181      0.856     -97.241     117.037
==============================================================================
Omnibus:                       13.911   Durbin-Watson:                   2.861
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               12.996
Skew:                           1.505   Prob(JB):                      0.00151
Kurtosis:                       4.984   Cond. No.                         9.34
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!   7   !!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.012
Model:                                 WLS   Adj. R-squared:                  0.002
Method:                      Least Squares   F-statistic:                    0.7096
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):              0.402
Time:                             19:26:31   Log-Likelihood:                -641.44
No. Observations:                      100   AIC:                             1287.
Df Residuals:                           98   BIC:                             1292.
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                74.0645     21.579      3.432      0.001      31.771     116.358
Q("BLACK + HISPANIC Stops - % Total")   -34.8557     41.378     -0.842      0.400    -115.954      46.243
==============================================================================
Omnibus:                      155.773   Durbin-Watson:                   1.820
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7540.533
Skew:                           5.746   Prob(JB):                         0.00
Kurtosis:                      43.959   Cond. No.                         8.16
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!   8   !!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.013
Model:                                 WLS   Adj. R-squared:                  0.011
Method:                      Least Squares   F-statistic:                     1.723
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):              0.190
Time:                             19:26:32   Log-Likelihood:                -2546.8
No. Observations:                      417   AIC:                             5098.
Df Residuals:                          415   BIC:                             5106.
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                79.9558     12.531      6.381      0.000      55.396     104.515
Q("BLACK + HISPANIC Stops - % Total")   -32.9364     25.092     -1.313      0.189     -82.116      16.244
==============================================================================
Omnibus:                      480.957   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32912.341
Skew:                           5.270   Prob(JB):                         0.00
Kurtosis:                      45.227   Cond. No.                         7.70
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!   9   !!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.042
Model:                                 WLS   Adj. R-squared:                  0.041
Method:                      Least Squares   F-statistic:                     9.775
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):            0.00180
Time:                             19:26:32   Log-Likelihood:                -11169.
No. Observations:                     1652   AIC:                         2.234e+04
Df Residuals:                         1650   BIC:                         2.235e+04
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                               154.5297     23.492      6.578      0.000     108.486     200.574
Q("BLACK + HISPANIC Stops - % Total")  -127.6918     40.843     -3.126      0.002    -207.742     -47.642
==============================================================================
Omnibus:                     2628.278   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1812252.529
Skew:                           9.824   Prob(JB):                         0.00
Kurtosis:                     164.065   Cond. No.                         7.38
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!   10   !!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.025
Model:                                 WLS   Adj. R-squared:                  0.024
Method:                      Least Squares   F-statistic:                     7.237
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):            0.00717
Time:                             19:26:32   Log-Likelihood:                -33782.
No. Observations:                     4625   AIC:                         6.757e+04
Df Residuals:                         4623   BIC:                         6.758e+04
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                               213.7977     40.392      5.293      0.000     134.630     292.966
Q("BLACK + HISPANIC Stops - % Total")  -176.0372     65.436     -2.690      0.007    -304.290     -47.785
==============================================================================
Omnibus:                    10015.008   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         56666231.998
Skew:                          19.173   Prob(JB):                         0.00
Kurtosis:                     543.908   Cond. No.                         7.27
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

## Stops vs accidents

In [125]:
for res in RESOLUTIONS:
    px.scatter(data_frame=grouped_dataframes2[res].query('Stops > 100'),
               x='Accident',
               y='Stops',
               size='Stops',
               color='Stops - % Total - WHITE',
               trendline='ols',
               title=res).show()

In [106]:
for res in RESOLUTIONS:
    px.scatter(data_frame=grouped_dataframes2[res],
               x='Stops per Accident',
               y='Stops - % Total - WHITE',
               size='Stops',
            #    color=grouped_dataframes2[res]['Stops - % Total - WHITE'],
               trendline='ols',
               title=res).show()

# Regressions

## Stops/Accident ~ Black+Hispanic % Total

In [ ]:
grouped_dataframes2[6].filter(like='BA')

In [133]:
for resol in RESOLUTIONS:

    mod = smf.wls(formula='''Q("Stops per Accident") ~ Q("BLACK + HISPANIC Stops - % Total")
                  ''', 
                  data=grouped_dataframes2[resol],
                  weights=grouped_dataframes2[resol]['Stops'])
    
    # WLS_R2_dict[resol] = mod.rsquared
    
    res = mod.fit(cov_type='HC3')
    print(f'!!!!!!!!!!!!!!       {resol}       !!!!!!!!!!!!!!')
    display(res.summary())

!!!!!!!!!!!!!!       6       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.006
Model:                                 WLS   Adj. R-squared:                 -0.039
Method:                      Least Squares   F-statistic:                   0.03279
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):              0.858
Time:                             19:22:19   Log-Likelihood:                -128.51
No. Observations:                       24   AIC:                             261.0
Df Residuals:                           22   BIC:                             263.4
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                47.5652     24.911      1.909      0.056      -1.259      96.389
Q("BLACK + HISPANIC Stops - % Total")     9.8978     54.664      0.181      0.856     -97.241     117.037
==============================================================================
Omnibus:                       13.911   Durbin-Watson:                   2.861
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               12.996
Skew:                           1.505   Prob(JB):                      0.00151
Kurtosis:                       4.984   Cond. No.                         9.34
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!!!!!!!!!!!!       7       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.012
Model:                                 WLS   Adj. R-squared:                  0.002
Method:                      Least Squares   F-statistic:                    0.7096
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):              0.402
Time:                             19:22:19   Log-Likelihood:                -641.44
No. Observations:                      100   AIC:                             1287.
Df Residuals:                           98   BIC:                             1292.
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                74.0645     21.579      3.432      0.001      31.771     116.358
Q("BLACK + HISPANIC Stops - % Total")   -34.8557     41.378     -0.842      0.400    -115.954      46.243
==============================================================================
Omnibus:                      155.773   Durbin-Watson:                   1.820
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7540.533
Skew:                           5.746   Prob(JB):                         0.00
Kurtosis:                      43.959   Cond. No.                         8.16
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!!!!!!!!!!!!       8       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.013
Model:                                 WLS   Adj. R-squared:                  0.011
Method:                      Least Squares   F-statistic:                     1.723
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):              0.190
Time:                             19:22:19   Log-Likelihood:                -2546.8
No. Observations:                      417   AIC:                             5098.
Df Residuals:                          415   BIC:                             5106.
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                79.9558     12.531      6.381      0.000      55.396     104.515
Q("BLACK + HISPANIC Stops - % Total")   -32.9364     25.092     -1.313      0.189     -82.116      16.244
==============================================================================
Omnibus:                      480.957   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32912.341
Skew:                           5.270   Prob(JB):                         0.00
Kurtosis:                      45.227   Cond. No.                         7.70
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!!!!!!!!!!!!       9       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.042
Model:                                 WLS   Adj. R-squared:                  0.041
Method:                      Least Squares   F-statistic:                     9.775
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):            0.00180
Time:                             19:22:19   Log-Likelihood:                -11169.
No. Observations:                     1652   AIC:                         2.234e+04
Df Residuals:                         1650   BIC:                         2.235e+04
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                               154.5297     23.492      6.578      0.000     108.486     200.574
Q("BLACK + HISPANIC Stops - % Total")  -127.6918     40.843     -3.126      0.002    -207.742     -47.642
==============================================================================
Omnibus:                     2628.278   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1812252.529
Skew:                           9.824   Prob(JB):                         0.00
Kurtosis:                     164.065   Cond. No.                         7.38
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

!!!!!!!!!!!!!!       10       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                               WLS Regression Results                              
===================================================================================
Dep. Variable:     Q("Stops per Accident")   R-squared:                       0.025
Model:                                 WLS   Adj. R-squared:                  0.024
Method:                      Least Squares   F-statistic:                     7.237
Date:                     Sun, 31 Oct 2021   Prob (F-statistic):            0.00717
Time:                             19:22:19   Log-Likelihood:                -33782.
No. Observations:                     4625   AIC:                         6.757e+04
Df Residuals:                         4623   BIC:                         6.758e+04
Df Model:                                1                                         
Covariance Type:                       HC3                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                               213.7977     40.392      5.293      0.000     134.630     292.966
Q("BLACK + HISPANIC Stops - % Total")  -176.0372     65.436     -2.690      0.007    -304.290     -47.785
==============================================================================
Omnibus:                    10015.008   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         56666231.998
Skew:                          19.173   Prob(JB):                         0.00
Kurtosis:                     543.908   Cond. No.                         7.27
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

## Stops ~ Accident + Race_i...

In [120]:
WLS_R2_dict = {}

for resol in RESOLUTIONS:

    mod = smf.wls(formula='''Stops ~ Accident
                  + Q("Stops - % Total - ASIAN")
                  + Q("Stops - % Total - BLACK")
                  + Q("Stops - % Total - HISPANIC")
                  + Q("Stops - % Total - OTHER")
                  ''', 
                  data=grouped_dataframes2[resol],
                  weights=grouped_dataframes2[resol]['Stops'])
    
    # WLS_R2_dict[resol] = mod.rsquared
    
    res = mod.fit(cov_type='HC3')
    print(f'!!!!!!!!!!!!!!       {resol}       !!!!!!!!!!!!!!')
    display(res.summary())

!!!!!!!!!!!!!!       6       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  Stops   R-squared:                       0.968
Model:                            WLS   Adj. R-squared:                  0.962
Method:                 Least Squares   F-statistic:                     7.783
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           9.58e-05
Time:                        19:07:03   Log-Likelihood:                -448.45
No. Observations:                  35   AIC:                             908.9
Df Residuals:                      29   BIC:                             918.2
Df Model:                           5                                         
Covariance Type:                  HC3                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                       -2.979e+04   3.83e+04     -0.778      0.436   -1.05e+05    4.52e+04
Accident                           33.7021      6.852      4.919      0.000      20.272      47.132
Q("Stops - % Total - ASIAN")     3.115e+05   1.05e+06      0.298      0.766   -1.74e+06    2.36e+06
Q("Stops - % Total - BLACK")    -1.535e+04   1.12e+05     -0.137      0.891   -2.35e+05    2.04e+05
Q("Stops - % Total - HISPANIC")  2.736e+05   2.77e+05      0.988      0.323   -2.69e+05    8.16e+05
Q("Stops - % Total - OTHER")    -4831.9285   1.02e+06     -0.005      0.996      -2e+06    1.99e+06
==============================================================================
Omnibus:                       15.283   Durbin-Watson:                   2.259
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.006
Skew:                           0.988   Prob(JB):                     2.25e-06
Kurtosis:                       6.732   Cond. No.                     9.32e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 9.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

!!!!!!!!!!!!!!       7       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  Stops   R-squared:                       0.790
Model:                            WLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     9.126
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           1.69e-07
Time:                        19:07:03   Log-Likelihood:                -1710.7
No. Observations:                 141   AIC:                             3433.
Df Residuals:                     135   BIC:                             3451.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                        3617.2358   6501.465      0.556      0.578   -9125.401    1.64e+04
Accident                           35.6096      7.150      4.981      0.000      21.597      49.623
Q("Stops - % Total - ASIAN")     2.161e+04   8.89e+04      0.243      0.808   -1.53e+05    1.96e+05
Q("Stops - % Total - BLACK")     2.701e+04    2.9e+04      0.931      0.352   -2.99e+04    8.39e+04
Q("Stops - % Total - HISPANIC")  5748.2255   3.86e+04      0.149      0.882   -6.99e+04    8.14e+04
Q("Stops - % Total - OTHER")    -7.394e+04   1.31e+05     -0.563      0.573   -3.31e+05    1.84e+05
==============================================================================
Omnibus:                       34.641   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              380.416
Skew:                           0.292   Prob(JB):                     2.48e-83
Kurtosis:                      11.026   Cond. No.                     9.73e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 9.73e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

!!!!!!!!!!!!!!       8       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  Stops   R-squared:                       0.819
Model:                            WLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     15.68
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           1.71e-14
Time:                        19:07:03   Log-Likelihood:                -6510.4
No. Observations:                 588   AIC:                         1.303e+04
Df Residuals:                     582   BIC:                         1.306e+04
Df Model:                           5                                         
Covariance Type:                  HC3                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         260.6080   1428.888      0.182      0.855   -2539.961    3061.177
Accident                           41.8894      5.891      7.111      0.000      30.344      53.435
Q("Stops - % Total - ASIAN")    -1.896e+04   1.71e+04     -1.112      0.266   -5.24e+04    1.45e+04
Q("Stops - % Total - BLACK")     1.177e+04    1.2e+04      0.979      0.327   -1.18e+04    3.53e+04
Q("Stops - % Total - HISPANIC") -4453.7895   1.24e+04     -0.358      0.720   -2.88e+04    1.99e+04
Q("Stops - % Total - OTHER")     7908.3834   1.61e+04      0.490      0.624   -2.37e+04    3.95e+04
==============================================================================
Omnibus:                      580.745   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           106912.483
Skew:                           3.815   Prob(JB):                         0.00
Kurtosis:                      68.617   Cond. No.                     1.92e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 1.92e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

!!!!!!!!!!!!!!       9       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  Stops   R-squared:                       0.461
Model:                            WLS   Adj. R-squared:                  0.460
Method:                 Least Squares   F-statistic:                     5.691
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           3.15e-05
Time:                        19:07:03   Log-Likelihood:                -27382.
No. Observations:                2706   AIC:                         5.478e+04
Df Residuals:                    2700   BIC:                         5.481e+04
Df Model:                           5                                         
Covariance Type:                  HC3                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         923.4690    527.564      1.750      0.080    -110.537    1957.475
Accident                           26.2748     11.932      2.202      0.028       2.889      49.661
Q("Stops - % Total - ASIAN")    -7181.5431   3475.055     -2.067      0.039    -1.4e+04    -370.561
Q("Stops - % Total - BLACK")     3596.2688   2353.232      1.528      0.126   -1015.982    8208.520
Q("Stops - % Total - HISPANIC")  -670.3989   2623.858     -0.256      0.798   -5813.067    4472.269
Q("Stops - % Total - OTHER")     -562.8997   2911.267     -0.193      0.847   -6268.879    5143.080
==============================================================================
Omnibus:                     3921.557   Durbin-Watson:                   1.783
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5037519.361
Skew:                           7.962   Prob(JB):                         0.00
Kurtosis:                     213.773   Cond. No.                     3.48e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 3.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

!!!!!!!!!!!!!!       10       !!!!!!!!!!!!!!


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  Stops   R-squared:                       0.212
Model:                            WLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     6.940
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           1.77e-06
Time:                        19:07:03   Log-Likelihood:                -99944.
No. Observations:               11153   AIC:                         1.999e+05
Df Residuals:                   11147   BIC:                         1.999e+05
Df Model:                           5                                         
Covariance Type:                  HC3                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                         458.6077    103.759      4.420      0.000     255.244     661.971
Accident                           14.0780      4.577      3.076      0.002       5.107      23.049
Q("Stops - % Total - ASIAN")    -1516.9424    357.373     -4.245      0.000   -2217.380    -816.505
Q("Stops - % Total - BLACK")      840.0243    400.223      2.099      0.036      55.603    1624.446
Q("Stops - % Total - HISPANIC")  -230.8678    417.046     -0.554      0.580   -1048.263     586.527
Q("Stops - % Total - OTHER")     -111.9758    411.750     -0.272      0.786    -918.992     695.040
==============================================================================
Omnibus:                    21971.627   Durbin-Watson:                   1.603
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         80846932.007
Skew:                          15.647   Prob(JB):                         0.00
Kurtosis:                     418.926   Cond. No.                         721.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Plotting

### Map

In [ ]:
def scatter_map(res:int):
    """Needs 'Latitude', 'Longitude' columns"""
    
    center_lat = df['Latitude'].median()
    center_long = df['Longitude'].median()

    fig = px.scatter_geo(grouped_dataframes[res], lon='Longitude', lat='Latitude', color='Stops per Accident')
    fig.update_layout(
            title = f'Stops per Accident - Resolution={res}',
            geo_scope='usa')
    fig.update_geos(
        center=dict(lon=center_long, lat=center_lat), 
        projection_scale=40
        )
    fig.show()

In [ ]:
for res in RESOLUTIONS:
    scatter_map(res=res)

In [ ]:
scatter_map(6)

In [ ]:
center_lat = df['Latitude'].median()
center_long = df['Longitude'].median()

fig = px.scatter_geo(grouped_dataframes[9], lon='Longitude', lat='Latitude', color='Stops per Accident')
fig.update_layout(
        title = f'Stops per Accident - {res}',
        geo_scope='usa')
fig.update_geos(
    center=dict(lon=center_long, lat=center_lat), 
    projection_scale=50
    )
fig.show()

### Unweighted

### Weighted

# Density Plots

In [121]:
len(df) / df['Accident'].sum()

48.04715099021946

In [ ]:
grouped_dataframes2[9]['Stops']

In [101]:
fig = px.histogram(data_frame=grouped_dataframes2[9][grouped_dataframes2[9]['Stops'] > 150],
                   histfunc="count",
                   x='Stops per Accident',
                   log_x=True,
                   color='Stops - % Total - WHITE',
                   nbins=600)
fig.show()

In [ ]:
fig = px.histogram(data_frame=grouped_dataframes[9][grouped_dataframes[9]['Stops'] > 50],
                   histfunc="count",
                   x='Stops per Accident',
                   nbins=600)
fig.show()

In [21]:
fig = make_subplots(cols=1, rows=len(grouped_dataframes),
                    shared_xaxes=True,
                    shared_yaxes=True,
                    vertical_spacing=.1)
hist_plot_dict = {}

for i,resolution in enumerate(RESOLUTIONS):
    # hist_plot_dict[resolution] = go.Histogram(x=grouped_dataframes[resolution]['Stops per Accident'])

    fig.append_trace(go.Histogram(histfunc="avg",
                                  x=grouped_dataframes[resolution]['Stops per Accident'],
                                  nbinsx=200,
                                  name=resolution),
                     col=1, row=i+1)

fig.update_xaxes(range=[0, 2000])
fig.update_yaxes(range=[0, 1000])
fig.update_layout(title_text="Customizing Subplot Axes", height=700, width=700)
fig.show()


In [ ]:
fig = go.Figure()
for resolution in RESOLUTIONS:
    fig.add_trace(go.Histogram(x=grouped_dataframes[resolution]['Stops per Accident']))
    
fig.show()

# QA

## Is #Stops correct?

In [ ]:
grouped_dataframes[9].loc['89f04280923ffff', 'Stops']==len(df[df['H3 Encoding - Res=9']=='89f04280923ffff'])

## Is #Arrests correct?

In [ ]:
grouped_dataframes[9].loc['89f042803abffff', 'Arrest'] == df[df['H3 Encoding - Res=9']=='89f04280923ffff']['Arrest'].sum()

## Are divisions being done correctly?

In [ ]:
sample_num_stops = len(df[df['H3 Encoding - Res=9']=='89f04280927ffff'])
sample_num_accidents = df[df['H3 Encoding - Res=9']=='89f04280927ffff']['Accident'].sum()

grouped_dataframes[9].loc['89f04280927ffff', 'Stops per Accident'] == sample_num_stops / sample_num_accidents

# CHECK EQUALITY ACROSS AGGREGATIONS

In [ ]:
for var in ['Stops', 'Citation', 'Accident']:
    for res in RESOLUTIONS:
        print(res, var, grouped_dataframes[res][var].sum())